# BDA03 — Tarea de Evaluación (Ecosistema Hadoop–Spark)
**Flujo ETL completo:** importación automática → exploración → limpieza (Pig) → transformación → carga en Hive → consultas HQL con JOIN.

**Dataset:** T1D (Guertin et al., 2024) — Virginia PrIMeD  
**Archivos:** `survey_data_and_results_final.xlsx` + `assay_final_genotyping_file.xlsx`  
**Clave de relación esperada (a validar):** `survey.SUBJECT_ID` ↔ `genotyping.FID`


## Explicación del conjunto de datos elegido
He elegido un dataset real sobre **riesgo genético de Diabetes Tipo 1 (T1D)** del estudio **Guertin et al. (2024)** (cohorte Virginia PrIMeD). Es interesante porque permite combinar datos **clínicos/demográficos** con datos **genéticos**, lo cual es un escenario típico de integración (JOIN) en Big Data.

### Archivos utilizados
Trabajo con **dos archivos interrelacionados**:

1) **`survey_data_and_results_final.xlsx` (Survey / Fenotipo)**  
Contiene información del participante (clínica/demográfica) y variables relacionadas con riesgo genético.  
**Clave:** `SUBJECT_ID`.

2) **`assay_final_genotyping_file.xlsx` (Genotyping / Genética)**  
Contiene marcadores genéticos (SNPs) por participante.  
**Clave:** `FID`.

### Relación entre archivos
La relación se realiza por el identificador de participante:
- **`survey.SUBJECT_ID = genotyping.FID`**  
(Validaré esta relación con datos reales antes de continuar.)

### Tamaño y relevancia
El dataset tiene en torno a **3.800 participantes**, suficiente para demostrar un flujo ETL completo con Hadoop (Pig) y Spark/Hive, incluyendo limpieza de datos, transformación y consultas con JOIN.


# Apartado 1: Importación automática + conversión a CSV


> Objetivo: traer los datos de forma reproducible (sin subida manual) y dejarlos en CSV para Pig/Spark/Hive.



In [1]:
# CELDA 2 — Apartado 1: Importación automática + conversión a CSV
# Objetivo: traer los datos de forma reproducible (sin subida manual) y dejarlos en CSV para Pig/Spark/Hive.

import os, shutil
import pandas as pd

# 1) Importación automática: clonar repo
!rm -rf BDA_dataset
!git clone https://github.com/kachytronico/BDA_dataset

# 2) Rutas reales a los XLSX dentro del repo clonado
base_path = "/content/BDA_dataset/Dataset not incorporated into the T1DKP"
survey_xlsx = os.path.join(base_path, "survey_data_and_results_final.xlsx")
geno_xlsx   = os.path.join(base_path, "assay_final_genotyping_file.xlsx")

assert os.path.exists(survey_xlsx), f"Falta archivo requerido: {survey_xlsx}"
assert os.path.exists(geno_xlsx),   f"Falta archivo requerido: {geno_xlsx}"

# 3) Copia a RAW (mantener originales intactos)
raw_dir = "/content/data/raw"
csv_dir = "/content/data/csv"
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(csv_dir, exist_ok=True)

survey_raw = os.path.join(raw_dir, "survey_data_and_results_final.xlsx")
geno_raw   = os.path.join(raw_dir, "assay_final_genotyping_file.xlsx")
shutil.copy2(survey_xlsx, survey_raw)
shutil.copy2(geno_xlsx, geno_raw)

# 4) Convertir a CSV (formato base para Pig/Spark/Hive)
df_survey = pd.read_excel(survey_raw)
df_geno   = pd.read_excel(geno_raw)

survey_csv = os.path.join(csv_dir, "survey.csv")
geno_csv   = os.path.join(csv_dir, "genotyping.csv")
df_survey.to_csv(survey_csv, index=False)
df_geno.to_csv(geno_csv, index=False)

# 5) Evidencia mínima visible
print("📁 Repo clonado (muestra):")
!ls -lh BDA_dataset | head -n 20

print("\n📁 RAW:")
!ls -lh /content/data/raw

print("\n📁 CSV:")
!ls -lh /content/data/csv

print("\nshape survey:", df_survey.shape)
print("shape genotyping:", df_geno.shape)

print("\nPreview survey (3 filas):")
display(df_survey.head(3))

print("\nPreview genotyping (3 filas):")
display(df_geno.head(3))


Cloning into 'BDA_dataset'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 64 (delta 8), reused 58 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 3.74 MiB | 14.96 MiB/s, done.
Resolving deltas: 100% (8/8), done.
📁 Repo clonado (muestra):
total 20K
drwxr-xr-x 3 root root 4.0K Feb 18 12:53 BDA03_cuadernos
drwxr-xr-x 2 root root 4.0K Feb 18 12:53 Dataset not incorporated into the T1DKP
drwxr-xr-x 4 root root 4.0K Feb 18 12:53 docs
-rw-r--r-- 1 root root 7.3K Feb 18 12:53 README.md

📁 RAW:
total 1.7M
-rw-r--r-- 1 root root 1.4M Feb 18 12:53 assay_final_genotyping_file.xlsx
-rw-r--r-- 1 root root 295K Feb 18 12:53 survey_data_and_results_final.xlsx

📁 CSV:
total 1.8M
-rw-r--r-- 1 root root 1.2M Feb 18 12:53 genotyping.csv
-rw-r--r-- 1 root root 528K Feb 18 12:53 survey.csv

shape survey: (3818, 15)
shape genotyping: (3818, 76)

Preview survey (3 filas):


,SUBJECT_ID,AGE,RACE,T1D_HIST,AUTO_HIST,AUTO_COND,AUTO_COND_4_TEXT,T1D_DIAG,T1D_DIAG_AGE,T1D_HOSP,DKA,GRS_HLA,GnonHLA,GRS,Risk
0,10011708520314,6,White,Yes,No,Not applicable,Not applicable,No,Not applicable,Not applicable,Not applicable,1.91,0.14,2.06,Not high
1,10021708520764,3,White,Don't know,Yes,"Thyroid_Hashimotos and_or Graves, Blood relati...",Not applicable,No,Not applicable,Not applicable,Not applicable,-1.41,1.93,0.52,Not high
2,10021708521587,7,Asian,Don't know,No,Not applicable,Not applicable,No,Not applicable,Not applicable,Not applicable,-13.66,0.77,-12.89,Not high



Preview genotyping (3 filas):


,FID,contact key,rs1049225,rs1052553,rs10795791,rs11203203,rs113010081,rs1150743,rs12416116,rs12720356,...,rs757411,rs7745656,rs7780389,rs917911,rs9268633,rs9271366,rs9273363,rs9357152,rs9469341,rs9585056
0,10011708520314,CONTACT10085,C:C,A:A,A:G,A:G,T:T,A:G,C:C,T:T,...,C:T,G:T,C:C,G:T,A:G,A:A,A:C,A:A,A:G,T:T
1,10021708520764,CONTACT14053,C:C,A:A,G:G,A:A,T:T,G:G,C:C,T:T,...,C:C,T:T,C:C,G:T,G:G,A:A,C:C,A:G,A:G,C:T
2,10021708521587,CONTACT11350,C:C,A:G,A:G,G:G,T:T,A:A,A:A,T:T,...,C:T,T:T,C:C,T:T,A:A,G:G,C:C,A:A,G:G,C:T


## Conclusiones (Apartado 1)
- He importado los datos de forma automática clonando el repositorio con `git clone`, sin subida manual.
- He conservado los originales en RAW (295 KB survey y 1.4 MB genotyping) y he convertido ambos a CSV (528 KB y 1.2 MB) para poder trabajar después con Pig y Spark/Hive.
- He verificado que ambos datasets tienen **3818 filas**; la estructura final es **15 columnas** en survey y **76 columnas** en genotyping.


# Apartado 2: Exploración con Pandas (usando CSV convertidos)



> Objetivo: validar JOIN real y detectar problemas de calidad (nulos, -9) sobre los CSV.



In [2]:
# CELDA 3 — Apartado 2: Exploración con Pandas (usando CSV convertidos)
# Objetivo: validar JOIN real y detectar problemas de calidad (nulos, -9) sobre los CSV.

import pandas as pd

survey_csv = "/content/data/csv/survey.csv"
geno_csv   = "/content/data/csv/genotyping.csv"

# 1) Cargar CSV forzando texto para evitar problemas de tipos
df_s = pd.read_csv(survey_csv, dtype=str)
df_g = pd.read_csv(geno_csv, dtype=str)

# 2) Confirmación de columnas clave
assert "SUBJECT_ID" in df_s.columns, "No existe SUBJECT_ID en survey"
assert "FID" in df_g.columns, "No existe FID en genotyping"

# 3) Validación JOIN real: survey.SUBJECT_ID = genotyping.FID
ids_s = df_s["SUBJECT_ID"].dropna().astype(str).str.strip()
ids_g = df_g["FID"].dropna().astype(str).str.strip()

print("IDs únicos survey (SUBJECT_ID):", ids_s.nunique())
print("IDs únicos genotyping (FID):  ", ids_g.nunique())
print("Intersección IDs:", len(set(ids_s).intersection(set(ids_g))))

m = pd.merge(
    df_s[["SUBJECT_ID"]],
    df_g[["FID"]],
    left_on="SUBJECT_ID",
    right_on="FID",
    how="inner"
)

print("Filas merge inner:", m.shape[0])
display(m.head(3))

# 4) Detección de nulos / vacíos (top 10 columnas)
print("\nNulos en survey (top 10 columnas):")
display(df_s.isna().sum().sort_values(ascending=False).head(10))

print("\nNulos en genotyping (top 10 columnas):")
display(df_g.isna().sum().sort_values(ascending=False).head(10))

# 5) Conteo de '-9' como missing codificado en genotyping
minus9_total = (df_g == "-9").sum().sum()
print("\nTotal de valores '-9' (string) en genotyping:", int(minus9_total))


IDs únicos survey (SUBJECT_ID): 3818
IDs únicos genotyping (FID):   3818
Intersección IDs: 3818
Filas merge inner: 3818


,SUBJECT_ID,FID
0,10011708520314,10011708520314
1,10021708520764,10021708520764
2,10021708521587,10021708521587



Nulos en survey (top 10 columnas):


,0
SUBJECT_ID,0
AGE,0
RACE,0
T1D_HIST,0
AUTO_HIST,0
AUTO_COND,0
AUTO_COND_4_TEXT,0
T1D_DIAG,0
T1D_DIAG_AGE,0
T1D_HOSP,0



Nulos en genotyping (top 10 columnas):


,0
FID,0
contact key,0
rs1049225,0
rs1052553,0
rs10795791,0
rs11203203,0
rs113010081,0
rs1150743,0
rs12416116,0
rs12720356,0



Total de valores '-9' (string) en genotyping: 181


## Conclusiones (Apartado 3 — Pandas)
- He confirmado que hay **3818 IDs únicos** en ambos archivos.
- La intersección de identificadores es total (**3818 IDs comunes**) y el `merge inner` devuelve **3818 filas**, validando la clave de unión **`survey.SUBJECT_ID = genotyping.FID`**.
- En el top 10 de columnas revisadas no he detectado valores nulos (`NaN`).


# Apartado 3: Localizar exactamente dónde está el missing codificado (-9)


> Objetivo: identificar qué columnas concretas contienen '-9' para justificar la limpieza en Pig.




In [3]:
# Apartado 3: Localizar exactamente dónde está el missing codificado (-9)
# Objetivo: identificar qué columnas concretas contienen '-9' para justificar la limpieza en Pig.

import pandas as pd

survey_csv = "/content/data/csv/survey.csv"
geno_csv   = "/content/data/csv/genotyping.csv"

df_s = pd.read_csv(survey_csv, dtype=str)
df_g = pd.read_csv(geno_csv, dtype=str)

# 1) Conteo por columna de '-9' en genotyping (missing codificado)
minus9_by_column = (df_g == "-9").sum()
minus9_by_column = minus9_by_column[minus9_by_column > 0].sort_values(ascending=False)

print("Columnas con '-9' (missing codificado) en genotyping:")
display(minus9_by_column)

print("\nTotal '-9' en genotyping:", int((df_g == "-9").sum().sum()))

# 2) Comprobación de cadenas vacías en survey (por si hubiese valores "" en vez de NaN)
empty_strings_survey = (df_s == "").sum()
empty_strings_survey = empty_strings_survey[empty_strings_survey > 0]

print("\nColumnas con cadenas vacías '' en survey:")
display(empty_strings_survey)


Columnas con '-9' (missing codificado) en genotyping:


,0
rs9585056,89
rs12927355,82
rs1367728,9
rs72727394,1



Total '-9' en genotyping: 181

Columnas con cadenas vacías '' en survey:


,0


## Conclusiones (calidad de datos)
- He detectado un total de **181 valores `"-9"`** en la tabla genotyping (missing codificado).
- El desglose es: `rs9585056` (89), `rs12927355` (82), `rs1367728` (9) y `rs72727394` (1).
- He comprobado que no existen cadenas vacías (`""`) en la tabla survey.
- En el siguiente apartado limpiaré estos `"-9"` convirtiéndolos a valores nulos reales para no distorsionar transformaciones y consultas.
